In [1]:
#code adapted from https://github.com/rbarreto/CLTC_InternetFragmentation/blob/master/Streaming/netflix_scrape_US.ipynb

In [64]:
#import packages
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml.html as lh
import regex as re
import numpy as np

In [30]:
#create a page from the link
page = requests.get("https://www.finder.com/a-z-of-international-netflix-tv-and-movie-lists")
doc = lh.fromstring(page.content)

In [31]:
#create a beautiful soup object of the page
soup = BeautifulSoup(page.content, "lxml")

In [32]:
#get all the links from the page
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))

In [33]:
#get all the links relating to movies and tv shows for a specific country
country_links = []
hashtag = False
for link in links:
    if link == '#tab5108':
        hashtag = False
    if hashtag:
        country_links.append(link) 
    if link == '#':
        hashtag = True
    
country_links = country_links[:-2]    

In [34]:
#separate the links into tv shows and movies
country_tv_show_links = []
country_movie_links = []
for link in country_links:
    if 'tv' in link:
        country_tv_show_links.append(link)
    if 'movie' in link:
        country_movie_links.append(link)

In [86]:
#create a dataframe of the tv shows for each country
country_tv_shows = pd.DataFrame()

k = 0
#loop through each country
for link in country_tv_show_links:
    country = []
    if k % 10 == 0:
        print(str(k) + "out of" + str(len(country_tv_show_links)))
    #create a page for each country
    country_page = requests.get(link)
    country_soup = BeautifulSoup(country_page.content, "lxml")
    #get the name of the country
    country_name = re.findall(r'(?<=Netflix\s).*(?=\sTV)', str(country_soup.title))
    if len(country_name)>0:
        country.append(country_name[0])
    country_name = re.findall(r'(?<=Netflix\sin\s).*(?=\<)', str(country_soup.title))
    if len(country_name)>0:
        country.append(country_name[0])
    country_name = re.findall(r'(?<=Current\s).*(?=\sNetflix)', str(country_soup.title))
    if len(country_name)>0:
        country.append(country_name[0])
       
    #create a doc for each country
    country_doc = lh.fromstring(country_page.content)
    tr_elements = country_doc.xpath('//tr')
    #Create empty list
    col=[]
    i=0
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
#         print('%d:"%s"'%(i,name))
        col.append((name,[]))
    
    
    #Since our first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=len(col):
            break
            

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
        
    #create a dataframe for each country
    Dict={title:column for (title,column) in col}
    country_df=pd.DataFrame(Dict)
    country_df["Country"] = country*len(country_df)  
    #add the country specific data frame to the overall dataframe
    country_tv_shows = pd.concat([country_tv_shows, country_df], sort=False) 
        
    k+=1
    

0out of32
10out of32
20out of32
30out of32


In [110]:
#make sure capture all countries
len(country_tv_show_links) == len(np.unique(country_tv_shows["Country"]))

True

In [167]:
#create a dataframe of the movies for each country
country_movies = pd.DataFrame()
#keep track of countries seen so far
countries = []
k = 0
#loop through each country
for link in country_movie_links:
    country = []
    if k % 10 == 0:
        print(str(k) + "out of" + str(len(country_movie_links)))
    
    country_names = []
    #create a page for each country
    country_page = requests.get(link)
    country_soup = BeautifulSoup(country_page.content, "lxml")
    
    #get the name of the country
    country_name = re.findall(r'(?<=\sin\s).*(?=\s\()', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    country_name = re.findall(r'(?<=Netflix\sin\sthe\s).*(?=\s\|)', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    country_name = re.findall(r'(?<=\sin\s).*(?=\<)', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    country_name = re.findall(r'(?<=\sNetflix\s).*(?=\sM)', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    country_name = re.findall(r'(?<=\sNetflix\s).*(?=\sm)', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    country_name = re.findall(r'(?<=Netflix\s).*(?=\sTV)', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    country_name = re.findall(r'(?<=\sin\s).*(?=\-)', str(country_soup.title))
    if len(country_name)>0:
        country_names.append(country_name[0])
    
    country.append(min(country_names))
    if country[0] in countries:
        continue
    countries.append(country[0])
    #create a doc for each country
    country_doc = lh.fromstring(country_page.content)
    tr_elements = country_doc.xpath('//tr')
    #Create empty list
    col=[]
    i=0
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
#         print('%d:"%s"'%(i,name))
        col.append((name,[]))
    
    
    #Since our first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=len(col):
            break
            

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
        
    #create a dataframe for each country
    Dict={title:column for (title,column) in col}
    country_df=pd.DataFrame(Dict)
    country_df["Country"] = country*len(country_df)  
    #add the country specific data frame to the overall dataframe
    country_movies = pd.concat([country_movies, country_df], sort=False) 

        
    k+=1
    

0out of34
10out of34
20out of34
30out of34


In [168]:
#make sure capture all countries 
#note that there is a duplicate link
len(country_movie_links) - 1 == len(np.unique(country_movies["Country"]))

True

In [172]:
#preview of tv shows
country_tv_shows.head()

,Title,Year of release,Genres,Watch it,Country,Runtime (mins),TV Show Name
0,A.D. Kingdom and EmpireSeason 1 (12 episodes),2015,TV Dramas Faith & Spirituality US TV Shows TV ...,Watch now,Algeria,NaN,NaN
1,Adventure TimeSeason 1 (17 episodes)Season 2 (...,2011,TV Animated Comedies TV Comedies TV Cartoons K...,Watch now,Algeria,NaN,NaN
2,After the RavesSeason 1 (9 episodes),2017,Music Social & Cultural Docs Documentaries Doc...,Watch now,Algeria,NaN,NaN
3,Age of TanksSeason 1 (4 episodes),2017,Military Documentaries Historical Documentarie...,Watch now,Algeria,NaN,NaN
4,All Hail King JulienSeason 1 (10 episodes)Seas...,2016,TV Animated Comedies TV Comedies TV Cartoons K...,Watch now,Algeria,NaN,NaN


In [174]:
country_tv_shows.tail()

,Title,Year of release,Genres,Watch it,Country,Runtime (mins),TV Show Name
1793,ŞubatSeason 1 (32 episodes),2013,TV Sci-Fi & Fantasy TV Mysteries TV Dramas Cri...,Watch now,US,NaN,NaN
1794,Ми-ми-мишкиSeason 1 (26 episodes),2018,Animation TV Cartoons Kids' TV Shows Animated ...,Watch now,US,NaN,NaN
1795,แผนร้ายนายเจ้าเล่ห์Season 1 (20 episodes),2016,TV Comedies Romantic TV Shows TV Shows Based o...,Watch now,US,NaN,NaN
1796,​SAINT SEIYA: Knights of the ZodiacOriginalSea...,2019,Action Anime Anime Series Anime Fantasy Anime ...,Watch now,US,NaN,NaN
1797,식샤를 합시다Season 1 (16 episodes),2013,TV Comedies Romantic TV Shows Korean TV Shows,Watch now,US,NaN,NaN


In [175]:
#preview of movies
country_movies.head()

,Title,Year of release,Runtime (mins),Genres,Watch it,Country
0,#Rucker50,2016,56.0,Documentaries,Watch now,Algeria
1,#realityhighOriginal,2017,99.0,Comedies,Watch now,Algeria
2,(T)ERROR,2015,84.0,Documentaries,Watch now,Algeria
3,1 Chance 2 Dance,2014,89.0,Dramas,Watch now,Algeria
4,1 Night,2016,75.0,Dramas,Watch now,Algeria


In [176]:
country_movies.tail()

,Title,Year of release,Runtime (mins),Genres,Watch it,Country
3881,Çok Filim Hareketler Bunlar,2010,99.0,Comedies,Watch now,US
3882,İstanbul Kırmızısı,2017,110.0,Dramas,Watch now,US
3883,​Goli Soda 2,2018,128.0,Action & Adventure,Watch now,US
3884,​Maj Rati ​​Keteki,2017,117.0,Dramas,Watch now,US
3885,​Mayurakshi,2017,100.0,Bengali-Language Movies,Watch now,US
